<a href="https://colab.research.google.com/github/o0nekov0o/OpenClassrooms_P2/blob/main/OpenClassrooms_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install virtualenv

!virtualenv OCP2_env

!source /content/OCP2_env/bin/activate; pip list

!source /content/OCP2_env/bin/activate; pip install requests

!source /content/OCP2_env/bin/activate; pip install bs4

!source /content/OCP2_env/bin/activate; pip install selenium

!source /content/OCP2_env/bin/activate; pip install webdriver-manager

!source /content/OCP2_env/bin/activate; pip list

!source /content/OCP2_env/bin/activate; pip freeze > OCP2_requirements.txt

!source /content/OCP2_env/bin/activate; pip install -r OCP2_requirements.txt

In [ ]:
!pip install -r OCP2_requirements.txt

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import os
import csv
import time

EXAMPLE_URL = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html' # global var
def get_book_data(EXAMPLE_URL):
    response = requests.get(EXAMPLE_URL)  # 200 si ok
    print(response)  # affichage code retour
    if response.ok:  # si justement ok
        soup = BeautifulSoup(response.text, features='html.parser')  # format txt, delete warnings
        """driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(EXAMPLE_URL)
        product_page_url = driver.current_url"""
        product_page_url = EXAMPLE_URL #soup.find pour nom classe exacte, soup.select nom classe inexaxte
        universal_product_code = soup.find('th', text='UPC').find_next_sibling('td').text  # colonne suivante, conversion txt
        title = soup.select('div.col-sm-6.product_main > h1')[0].text # unique indice, conversion txt
        price_excluding_tax = soup.find('th', text='Price (excl. tax)').find_next_sibling('td').text.replace('Â', '')  # comme title, enleve specialchar
        price_including_tax = soup.find('th', text='Price (incl. tax)').find_next_sibling('td').text.replace('Â', '')  # comme title, enleve specialchar
        number_available = soup.find('th', text='Availability').find_next_sibling('td').text  # comme title, conversion txt
        product_description = soup.find('div', {'id': 'product_description'}).find_next_sibling('p').text  # comme title, balises differentes
        category = soup.find('li', {'class': 'active'}).find_previous_sibling('li').find('a').text  # inverse title, balises differentes, recuperation lien
        review_rating = f"{soup.select('p.star-rating')[0]['class'][1]} Star(s) on Five" # print interpreter, comportement code, liste dans liste
        image_url = soup.select('div.item.active > img')[0]['src'].replace('../..', 'http://books.toscrape.com') # comme review_rating, liste attribut src
        print(f"Product Page URL : {product_page_url}") # fprint on evite, + et \n, code plus lisible
        print(f"UPC: {universal_product_code}")
        print(f"Title: {title}")
        print(f"Price Excluding Tax: {price_excluding_tax}")
        print(f"Price Including Tax: {price_including_tax}")
        print(f"Number Available: {number_available}")
        print(f"product_description: {product_description}")
        print(f"category: {category}")
        print(f"Review Rating: {review_rating}")
        print(f"Image URL: {image_url}")
        print('―' * 100) # ligne separation
        save_image_file(image_url) # appel fonction, sauvegarde image
        return {'Product Page URL': product_page_url, 'UPC': universal_product_code, 'Title': title, # strings headers, vars contents
                'Price Excluding Tax': price_excluding_tax, 'Price Including Tax': price_including_tax, # pour ecrire contenu sous headers correspondants
                'Number Available': number_available, 'Product Description': product_description,
                'Category': category, 'Review Rating': review_rating, 'Image URL': image_url}

def save_book_csv(book_data):
    file_exists = os.path.isfile(f"scraping_{list(book_data.values())[7].replace(' ','_')}.csv") # permettra verifier si csv existe
    file = open(f"scraping_{list(book_data.values())[7].replace(' ','_')}.csv", 'a', encoding='utf-8') # retour get_book_data, value index 7, cat name
    headers = list(book_data) # retourne clés book_data dans liste pour headers
    writer = csv.DictWriter(file, fieldnames=headers) #preparation ecriture headers
    if not file_exists:
        writer.writeheader() # si fichier n'existe pas, ecrire headers creation
    writer.writerow(book_data) # ecrire contenu sous headers correspondants
    file.close()

index_url = 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html' # global var
def save_one_category(index_url):
    """
    Save all books of a category (it handles pagination)
    """
    suivant = 1 # permettra de relancer la boucle
    url = index_url # preserve global var
    while suivant == 1: #condition relance boucle
        response = requests.get(url)  # 200 si ok
        print(response)  # afficher code retour
        if response.ok:  # si justement ok
            soup = BeautifulSoup(response.text, features='html.parser') # format txt, delete warnings
            for i in soup.find_all('div', {'class': 'image_container'}): # plusieurs classes image_container
                lien = i.find('a')['href'].replace('../../..','http://books.toscrape.com/catalogue') # recup lien
                book_data = get_book_data(lien) # appel fonction get_book_data, retour dans book_data           
                save_book_csv(book_data) # appel foncion save_book_csv, enregistrer valeurs retournées
                time.sleep(0.25) # meilleure lecture dans interpreter, si corrections bugs
            if soup.find('li', {'class': 'next'}): # si presence bouton suivant
                base_url = url[:url.rfind('/')] # sup derniere partie url (initialement index)
                suffix_url = soup.find('li', {'class': 'next'}).find('a')['href'] # isole lien bouton suivant
                url = f"{base_url}/{suffix_url}" # ajout lien bouton suivant au reste
                # url = f"{url[:url.rfind('/')]}/{soup.find('li', {'class': 'next'}).find('a')['href']}"
                suivant = 1 # relancer la boucle (car bouton suivant present)
            else:
                suivant = 0 # sortir de la boucle (car bouton suivant absent)
    
site_url = 'http://books.toscrape.com/index.html' # global var
def save_all_categories(site_url):
    curl = site_url # preserve global var
    response = requests.get(curl)  # 200 si ok
    print(response)  # afficher code retour
    if response.ok:  # si justement ok
        soup = BeautifulSoup(response.text, features='html.parser') # format txt, delete warnings
        for x in soup.find_all('ul', {'class': 'nav nav-list'}): # unique classe 'nav nav-list' mais find_all iteration
            links = x.find('ul').find_all('a') # autre ul, lister tous les lien des iterations
        for n in range(len(links)): # parcourir toute la liste de liens récupérés
            link = links[n]['href'].replace('catalogue', 'http://books.toscrape.com/catalogue') # affecter dans variable
            try:
                save_one_category(link) # même variable pour appeler fonction
            except AttributeError: # si description absente
                continue
            time.sleep(0.25)

class save_image_global: # pour créer fonction global
    global save_image_file # car appelé avant par get_book_data
    def save_image_file(url_image):
        if os.path.isdir('scraping_images') == False: # si le dossier n'existe pas 
            os.makedirs('scraping_images') # alors procéder à sa création
        img_url = url_image.replace('http://books.toscrape.com/media/cache/','').replace('/','_') # preparer nom image
        img_data = requests.get(url_image).content # preparer contenu image
        with open(f'scraping_images/{img_url}', 'wb') as handler: # from prepared image name
            handler.write(img_data) # ecrire contenu image (dessiner l'img)

save_all_categories(site_url)

if __name__ == "main":
    book_data = get_book_data(EXAMPLE_URL)
    save_book_csv(book_data)
    save_one_category(index_url)
    save_all_categories(site_url)
    save_image_file(url_image)

In [ ]:
import shutil
import glob

reponse = 'o'
while not reponse == 'y' or 'n':
    reponse = input('Reset the repertory, yes (y) or no (n) ? ')
    if reponse == 'y':
        shutil.rmtree('/content/scraping_images')
        for filename in glob.glob('/content/scraping_*'):
            os.remove(filename)
    elif reponse == 'n':
        break